In [1]:
#Celda para librerías


import sklearn as sk

import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression



print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())



pd.set_option('display.max_columns', None)
#pd.set_option('display_max_rows', 100)

1.23.5
Directorio actual: c:\Users\Yeray\Desktop\DATA_SCIENCE_ML\TFG\NOTEBOOKS


In [2]:
#Error al leer solucionado asi v

#The error you're encountering is due to the backslashes in the file path being interpreted 
# as escape characters. To fix this, you can either use raw strings by prefixing the string 
# with an r, or you can replace the backslashes with forward slashes.

url = r"C:\Users\Yeray\Desktop\DATA_SCIENCE_ML\TFG\DATA\home-credit-default-risk\application_train.csv"

df = pd.read_csv(url)
#df.info() #Tenemos 122 columnas, 65 tipo float, 41 tipo int y 16 tipo object
descripcion = df.describe() 
#print(descripcion)

#Vamos a ver si hay valores nulos
"""
missing_values = df.isnull().sum().sort_values(ascending = False)

missing_values_df = missing_values[missing_values > 0]  # Filtra solo las columnas con missing values
missing_values_df= missing_values_df.to_frame()  # Convierte la serie en un dataframe
missing_values_df = missing_values_df.T  # Transpone la tabla para que las columnas se muestren como filas
missing_values_df = missing_values_df.reset_index(drop=True)  # Resetea el índice

print(missing_values_df.to_string(index=False))  # Muestra la tabla sin índices
"""

#Vamos a tratar estos valores nulos
#Primero clasificamos las columnas por tipo de datos
#Columnas tipo numérico
numeric_columns = df.select_dtypes(include=[np.number]).columns
#Columnas tipo categóricas
categoric_columns = df.select_dtypes(include=[object]).columns

#Vamos a tratar los valores nulos de las columnas numéricas rellenando con la mediana
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median()) 
#Vamos a tratar los valores nulos de las columnas categóricas rellenando con 0, que representa un valor desconocido
df[categoric_columns] = df[categoric_columns].fillna(0)
print(df.isnull().sum().sort_values(ascending = False))  #Comprobamos que ya no hay valores nulos

dfnomsv=df.copy()


SK_ID_CURR                     0
OBS_30_CNT_SOCIAL_CIRCLE       0
WALLSMATERIAL_MODE             0
TOTALAREA_MODE                 0
HOUSETYPE_MODE                 0
                              ..
LIVE_REGION_NOT_WORK_REGION    0
REG_REGION_NOT_WORK_REGION     0
REG_REGION_NOT_LIVE_REGION     0
HOUR_APPR_PROCESS_START        0
AMT_REQ_CREDIT_BUREAU_YEAR     0
Length: 122, dtype: int64


In [3]:

#Variables categoricas a cuantitativas
# Identificar columnas de tipo object
object_columns = dfnomsv.select_dtypes(include=['object']).columns
print("Columnas tipo object:", object_columns)

#Vamos a ver si podemos convertir las columnas object a numéricas
#En este caso no es posible convertir ninguna columna object a numérica
""""
for col in object_columns:
    try:
        df[col] = pd.to_numeric(df[col])
        print(f"Columna {col} convertida a numérica")
    except:
        print(f"No se pudo convertir la columna {col} a numérica")
"""
#Vamos a inspeccionar las columnas que han sido no convertibles para analizarlas más a fondo
for col in object_columns:
    if not pd.api.types.is_numeric_dtype(dfnomsv[col]):
        print(f"Valores único en la columna {col}:")
        print(dfnomsv[col].unique())
        
#Codificación one-hot de las columnas categóricas (pd.getdummies)
#df2 = pd.get_dummies(df2, columns=object_columns, drop_first=True)
#El problema de esto es que nos genera un dataframe con 246 columnas por lo que no lo veo rentable.



Columnas tipo object: Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')
Valores único en la columna NAME_CONTRACT_TYPE:
['Cash loans' 'Revolving loans']
Valores único en la columna CODE_GENDER:
['M' 'F' 'XNA']
Valores único en la columna FLAG_OWN_CAR:
['N' 'Y']
Valores único en la columna FLAG_OWN_REALTY:
['Y' 'N']
Valores único en la columna NAME_TYPE_SUITE:
['Unaccompanied' 'Family' 'Spouse, partner' 'Children' 'Other_A' 0
 'Other_B' 'Group of people']
Valores único en la columna NAME_INCOME_TYPE:
['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']
Valores único en la columna NAME_EDU

In [4]:
#Vamos a convertirlas mediante LabelEncoder
label_encoder = LabelEncoder()
#Convertimos previamente todos los valores de las columnas object a string para que labelencoder pueda trabajar con ellas
for col in object_columns:
    dfnomsv[object_columns] = dfnomsv[object_columns].astype(str)
    dfnomsv[col] = label_encoder.fit_transform(dfnomsv[col])
    print(f"Columna {col} convertida a numérica mediante LabelEncoder")

Columna NAME_CONTRACT_TYPE convertida a numérica mediante LabelEncoder
Columna CODE_GENDER convertida a numérica mediante LabelEncoder
Columna FLAG_OWN_CAR convertida a numérica mediante LabelEncoder
Columna FLAG_OWN_REALTY convertida a numérica mediante LabelEncoder
Columna NAME_TYPE_SUITE convertida a numérica mediante LabelEncoder
Columna NAME_INCOME_TYPE convertida a numérica mediante LabelEncoder
Columna NAME_EDUCATION_TYPE convertida a numérica mediante LabelEncoder
Columna NAME_FAMILY_STATUS convertida a numérica mediante LabelEncoder
Columna NAME_HOUSING_TYPE convertida a numérica mediante LabelEncoder
Columna OCCUPATION_TYPE convertida a numérica mediante LabelEncoder
Columna WEEKDAY_APPR_PROCESS_START convertida a numérica mediante LabelEncoder
Columna ORGANIZATION_TYPE convertida a numérica mediante LabelEncoder
Columna FONDKAPREMONT_MODE convertida a numérica mediante LabelEncoder
Columna HOUSETYPE_MODE convertida a numérica mediante LabelEncoder
Columna WALLSMATERIAL_MODE 

In [5]:
#Mapa de calor de correlaciones respecto a TARGET para ver la importancia y relaciones de las variables

report = ProfileReport(dfnomsv, minimal=True)
#display(report)
#report.to_file("report_minimal.html")

corr_matriz_con_TARGET = dfnomsv.corr()['TARGET'].sort_values(ascending=False)
print(corr_matriz_con_TARGET)

TARGET                         1.000000
DAYS_BIRTH                     0.078239
REGION_RATING_CLIENT_W_CITY    0.060893
REGION_RATING_CLIENT           0.058899
DAYS_LAST_PHONE_CHANGE         0.055218
                                 ...   
AMT_GOODS_PRICE               -0.039623
DAYS_EMPLOYED                 -0.044932
EXT_SOURCE_1                  -0.098887
EXT_SOURCE_3                  -0.155892
EXT_SOURCE_2                  -0.160295
Name: TARGET, Length: 122, dtype: float64


In [ ]:
#Además de la correlación, vamos a ver la importancia de las variables mediante algunos modelos de ML	
X = dfnomsv.drop('TARGET', axis=1)
y= dfnomsv['TARGET']

#Entrenamiento del modelo
model = RandomForestClassifier(random_state=42, n_jobs=-1) 
model.fit(X, y) 

#Obtener importancia de las característica
importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print(importances)

important_features = importances[importances > 0]
X_filtrada = X[important_features.index]

print(f"Dimensiones originales: {X.shape}")
print(f"Dimensiones después de filtrar características importantes: {X_filtrada.shape}")

#Vamos a eliminar las columnas que mayoritariamente tienen 0's
zero_threshhold = 0.95
high_zero_cols = [col for col in X_filtrada.columns if (X_filtrada[col] == 0).mean() > zero_threshhold]
X_filtrada= X_filtrada.drop(columns=high_zero_cols)

print(f"Dimensiones después de eliminar columnas con más del 95% ceros: {X_filtrada.shape}")

#Division train/test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Normalizamos los datos
scaler = StandardScaler()
x_train_escalada = scaler.fit_transform(x_train)
x_test_escalada = scaler.transform(x_test)

#Random Forest
modelrf = RandomForestClassifier(random_state=42, n_jobs=-1)
modelrf.fit(x_train_escalada, y_train)
y_pred = modelrf.predict(x_test_escalada)
roc_auc_rf = roc_auc_score(y_test, y_pred)  
print(f"ROC AUC del modelo de Random Forest: {roc_auc_rf}")

#Regresión Logística
modellr = LogisticRegression(random_state=42, max_iter=1000)
modellr.fit(x_train_escalada, y_train)
y_pred_lr = modellr.predict_proba(x_test_escalada)[:, 1]
roc_auc_lr = roc_auc_score(y_test, y_pred_lr)
print(f"Logistic Regression ROC-AUC: {roc_auc_lr:.4f}")

# Validación cruzada para Random Forest
cv_scores_rf = cross_val_score(modelrf, x_train_escalada, y_train, cv=5, scoring='roc_auc')
print(f"Random Forest CV ROC-AUC: {np.mean(cv_scores_rf):.4f} ± {np.std(cv_scores_rf):.4f}")

# Validación cruzada para Regresión Logística
cv_scores_lr = cross_val_score(modellr, x_train_escalada, y_train, cv=5, scoring='roc_auc')
print(f"Logistic Regression CV ROC-AUC: {np.mean(cv_scores_lr):.4f} ± {np.std(cv_scores_lr):.4f}")

#Características más importantes
print("Características más importantes seleccionadas:")
print(important_features.sort_values(ascending=False).head(10))
